In [1]:
import torch
import ssl
import matplotlib.pyplot as plt
from src.logger.logger_initializer import LoggerInitializer

ssl._create_default_https_context = ssl._create_unverified_context
device = 'cuda'
torch.__version__

LoggerInitializer().init()

[2023-04-17 01:26:19] 28766 root {logger_initializer-47} INFO - Current process ID: 28766


True

In [2]:
from src.data_loader.dataset_getter import DatasetGetter
dataset = DatasetGetter('../../bayesian_test/data', batch_size=12).get()

In [3]:
from src.models.bnn.pyro_resnet18_bnn_classifier import PyroResnet18BnnClassifier

classifier = PyroResnet18BnnClassifier(dataset=dataset, device=device).init()

In [4]:
result_df, result_dict = classifier.fit(num_epoch=2)
display(result_df)
print(result_dict)

[2023-04-17 01:26:46] 28766 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 0/2]: f1_score: 0.149 accuracy_score: 0.219 loss: 17506354108.217 
[2023-04-17 01:27:09] 28766 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 1/2]: f1_score: 0.127 accuracy_score: 0.248 loss: 16940679741.715 


,f1_score,accuracy_score,loss
0,0.148712,0.218750,1.750635e+10
1,0.126871,0.247656,1.694068e+10


{'best_loss': 16940679741.715069, 'best_f1_score': 0.14871243564446257, 'best_accuracy_score': 0.24765625, 'best_loss_model': OrderedDict([('_resnet.bn1.weight', tensor([ 2.3210e-01,  2.4354e-01, -5.1096e-08,  5.0026e-01,  3.4404e-09,
         2.1195e-01,  4.2739e-01, -7.6317e-04,  2.8591e-01,  5.1888e-04,
         3.3121e-01,  2.2828e-01,  3.8649e-01,  6.3212e-05,  3.0997e-01,
         2.3101e-01,  2.6248e-01,  3.8151e-01,  4.6948e-01,  2.7845e-01,
         2.6138e-01,  3.1442e-01,  2.8271e-01,  2.0901e-01,  2.2784e-01,
         2.7008e-01,  3.0376e-01,  3.3140e-01,  3.7723e-01,  3.1927e-01,
         2.8154e-01,  2.1673e-01,  3.1126e-01,  3.3585e-01,  4.2321e-01,
         3.7645e-01, -8.6544e-03,  1.6641e-01,  6.2444e-04,  2.1714e-01,
         1.7046e-01,  2.2691e-01,  2.9030e-01,  2.3331e-01,  2.9787e-01,
         3.4079e-01,  2.3965e-01,  2.7257e-01,  2.2001e-08,  2.6743e-01,
         2.1210e-01,  2.8783e-01,  3.2559e-01,  2.3103e-01,  3.5917e-01,
         1.9895e-01,  2.5746e-01,  

In [5]:
classifier.save_model(result_dict)

In [6]:
test_model_weights_path = '../../bayesian_refactoring/assets/model_weights/pyro_resnet18bnn_28766_97cd2c3b-14db-49e6-9035-0474b00c51be.pt'

In [7]:
test_results = torch.load(test_model_weights_path)

In [8]:
new_classifier = PyroResnet18BnnClassifier(
    dataset=dataset, device=device
).load_model(
    model_state_dict=test_results['best_f1_score_model'],
    pyro_state_dict=test_results['pyro_state_dict']
)

In [9]:
new_classifier.score(dataset.validate_dataloader)

{'f1_score': 0.11227969435503775, 'accuracy_score': 0.22924901185770752}